# MoleculeNet - HIV

HIV dataset: "Experimentally measured abilities to inhibit HIV replication".
The dataset is part of the MoleculeNet benchmark suite.

From TUDataset benchmark:  https://moleculenet.org/datasets-1

You can find similar projects on github, youtube or scholar searching for "molecular property prediction"

## IMPORTANT NOTES
- It's a classification problem, the evaluation metric is ROC-AUC.

- The dataset is very unbalanced, so try to explore possible strategies to handle it.

- The prescribed dataset train/val/test split is Scaffold. This is a specific strategy to divide molecules in clusters that present similar molecular structure or function. We train only on a few clusters and we evaluate on different ones. It is basically forcing out of distribution inference. If you want an additional challenge, you can try to implement it, it will force you to use the popular chemical library 'rdkit'.

- You shuold be able to use colab/kaggle to train a model on gpu, you can also create a smaller version of the dataset if you have constraints on the resources.

In [1]:
import torch
from torch_geometric.datasets import MoleculeNet
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, GATConv, GINConv, GINEConv
from torch.nn import Sequential, Linear, ReLU
from torch.optim.lr_scheduler import ReduceLROnPlateau

/home/leonardodg/code/GML-Fall25/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load HIV datasets from MoleculeNet benchmark suite
dataset = MoleculeNet(root='./data', name='HIV')

Processing...
/home/leonardodg/code/GML-Fall25/.venv/lib/python3.12/site-packages/torch_geometric/data/dataset.py:265: UserWarning: Skipping molecule 'O=C1O[Al]23(OC1=O)(OC(=O)C(=O)O2)OC(=O)C(=O)O3' since it resulted in zero atoms
  self.process()
/home/leonardodg/code/GML-Fall25/.venv/lib/python3.12/site-packages/torch_geometric/data/dataset.py:265: UserWarning: Skipping molecule 'Cc1ccc([B-2]2(c3ccc(C)cc3)=NCCO2)cc1' since it resulted in zero atoms
  self.process()
/home/leonardodg/code/GML-Fall25/.venv/lib/python3.12/site-packages/torch_geometric/data/dataset.py:265: UserWarning: Skipping molecule 'Oc1ccc(C2Oc3cc(O)cc4c3C(=[O+][AlH3-3]35([O+]=C6c7c(cc(O)cc7[OH+]3)OC(c3ccc(O)cc3O)C6O)([O+]=C3c6c(cc(O)cc6[OH+]5)OC(c5ccc(O)cc5O)C3O)[OH+]4)C2O)c(O)c1' since it resulted in zero atoms
  self.process()
/home/leonardodg/code/GML-Fall25/.venv/lib/python3.12/site-packages/torch_geometric/data/dataset.py:265: UserWarning: Skipping molecule 'CC1=C2[OH+][AlH3-3]34([O+]=C2C=CN1C)([O+]=C1C=CN(C)C(

In [3]:
print(f"Example of a graph:\n {dataset[0]}\n")
print(f"Node features:\n {dataset.x}\n")
print(f"Edge features:\n {dataset.edge_attr}\n")
print(f"Graph labels:\n {dataset.y}")

Example of a graph:
 Data(x=[19, 9], edge_index=[2, 40], edge_attr=[40, 3], smiles='CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)=[O+]2', y=[1, 1])

Node features:
 tensor([[6, 0, 4,  ..., 4, 0, 0],
        [6, 0, 4,  ..., 4, 0, 0],
        [6, 0, 3,  ..., 3, 0, 1],
        ...,
        [7, 0, 2,  ..., 3, 1, 1],
        [8, 0, 2,  ..., 3, 1, 1],
        [6, 0, 3,  ..., 3, 1, 1]])

Edge features:
 tensor([[ 1,  0,  0],
        [ 1,  0,  0],
        [ 1,  0,  0],
        ...,
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1]])

Graph labels:
 tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])


# Hints
- you may need a dataloader since the dataset is quite large
- you are free to proceed with any architecture, supervised / unsupervised / self-supervised
- you can find multiple examples of similar tasks or model on github